In [135]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
from sklearn.metrics import accuracy_score

In [136]:
class Classifier(nn.Module):
    """
    Our simple classifier
    """
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)	
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [137]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize([0.5], [0.5]),
                              ])

In [138]:
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)

In [139]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [142]:
# define model, loss function and optimizer
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

In [143]:
# hyperparameters
epochs = 5

In [151]:
# training loop
model.train()
for e in range(epochs):
    train_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
        
        optimizer.zero_grad()
        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    else:
        print(f"Training loss: {train_loss/len(trainloader)}")

Training loss: 0.5764774013874627
Training loss: 0.5452300261205701
Training loss: 0.5208178315081322
Training loss: 0.5010533543315523
Training loss: 0.48531240132698883


In [152]:
# load validation data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [153]:
# get validation prediction
model.eval()

y_preds = np.empty((1))
y_test = np.empty((1))
for images, labels in testloader:
    with torch.no_grad():
        images = images.view(images.shape[0], -1)
        output = model.forward(images)
        y_test = np.concatenate([y_test,labels.numpy()], axis=0)
        _, top_class = output.topk(1, dim=1)
        y_preds = np.concatenate([y_preds,np.squeeze(top_class.numpy())], axis=0)

In [155]:
# get validation score of our model
y_test = y_test.astype(int)
y_preds = y_preds.astype(int)
accuracy_score(y_test, y_preds)

In [158]:
# save the model state
torch.save(model.state_dict(), 'linear_mnist.pth')

In [160]:
# run a tracing module to convert to Torch Script
example_image, example_label = next(iter(trainloader))

In [161]:
traced_script_module = torch.jit.trace(model, example_image)

In [163]:
traced_script_module.save("traced_linear_mnist.pt")